In [256]:
import pandas as pd
import re
import string
from spacy.lang.de.stop_words import STOP_WORDS
# Spacy
import spacy

In [257]:
pd.set_option('display.max_colwidth', None)
nlp = spacy.load('de_core_news_md')

In [258]:
df = pd.read_csv('data/reviews.csv', sep=';')

In [259]:
# Remove NaN caption 
df = df.loc[~df['caption'].isnull(), ['caption', 'rating', 'timestamp']]
df.reset_index(drop=True)
df.head(3)

,caption,rating,timestamp
0,"Die mit Abstand leckerste Pizza in Deutschland, und das selbst wenn man sie um halb 10 abends zum mitnehmen bestellt :) Einfach zum niederknien! …",5.0,2023-02-15 22:10:07.146586
4,Good place to dine in! Sadly no chicken pizzas available. Do keep in mind the waiting time when you visit✌🏻 …,4.0,2023-02-14 00:10:07.146703
5,This is the must try pizza in Frankfurt.,5.0,2023-02-14 00:10:07.146705


In [260]:
# Remove the section translated by google if present
df['caption'] = df['caption'].str.split('(Translated by Google)').str[0]
df.head(3)

,caption,rating,timestamp
0,"Die mit Abstand leckerste Pizza in Deutschland, und das selbst wenn man sie um halb 10 abends zum mitnehmen bestellt :) Einfach zum niederknien! …",5.0,2023-02-15 22:10:07.146586
4,Good place to dine in! Sadly no chicken pizzas available. Do keep in mind the waiting time when you visit✌🏻 …,4.0,2023-02-14 00:10:07.146703
5,This is the must try pizza in Frankfurt.,5.0,2023-02-14 00:10:07.146705


In [261]:
print (STOP_WORDS)

{'nahm', 'währenddem', 'darin', 'solchem', 'so', 'vierte', 'zweiter', 'demgemäß', 'gehen', 'zehnter', 'heute', 'wollen', 'sollte', 'en', 'dahin', 'derjenigen', 'über', 'eigenes', 'bekannt', 'konnte', 'beim', 'du', 'besser', 'ins', 'vom', 'mancher', 'sechstes', 'daß', 'ganz', 'muss', 'sollen', 'deswegen', 'wollt', 'dein', 'ihres', 'gedurft', 'gesagt', 'seitdem', 'deren', 'machen', 'recht', 'die', 'mussten', 'dort', 'seiner', 'lang', 'zusammen', 'dritte', 'sei', 'nein', 'na', 'noch', 'wart', 'man', 'wo', 'indem', 'uhr', 'gekannt', 'eines', 'wohl', 'achtes', 'einiger', 'den', 'werden', 'großen', 'zehnte', 'gleich', 'wirklich', 'dieselbe', 'besten', 'siebentes', 'acht', 'weiteres', 'statt', 'aus', 'tage', 'sonst', 'keinen', 'zeit', 'offen', 'weniger', 'zum', 'durfte', 'dafür', 'nachdem', 'solcher', 'derjenige', 'siebter', 'oben', 'mir', 'siebten', 'einem', 'leider', 'nie', 'tel', 'mein', 'hier', 'konnten', 'ohne', 'sowie', 'jedem', 'genug', 'meines', 'erster', 'wie', 'eben', 'damit', 'zurü

In [262]:
REGX_USERNAME = r"@[A-Za-z0-9$-_@.&+]+"
REGX_URL = r"https?://[A-Za-z0-9./]+"
def preprocessing(text):
    text = text.lower()

    # Remove
    text = re.sub(REGX_USERNAME, ' ', text)
    text = re.sub(REGX_URL, ' ', text)

    # Replace Emojis
    emojis = {
        ':)': 'positive emotionen',
        ':(': 'negative emotionen'
    }

    for e in emojis:
        text = text.replace(e, emojis[e])

    tokens = [token.text for token in nlp(text)]

    # Remove Stop Words, punctuations and words with less than 3 characters
    tokens = [t for t in tokens if
              t not in STOP_WORDS and
              t not in string.punctuation and
              len(t) > 3]

    tokens = [t for t in tokens if not t.isdigit()]

    text = " ".join(tokens);

    # Lemmatization
    allowed_postags = ["NOUN", "ADJ", "VERB", "ADV"]
    doc = nlp(text)
    new_text = []
    for token in doc:
        if token.pos_ in allowed_postags:
            new_text.append(token.lemma_)
    lemmatized = " ".join(new_text).lower()

    return lemmatized

df["caption_clean"] = df["caption"].apply(preprocessing)
df[['caption', 'caption_clean', 'rating']].head()

,caption,caption_clean,rating
0,"Die mit Abstand leckerste Pizza in Deutschland, und das selbst wenn man sie um halb 10 abends zum mitnehmen bestellt :) Einfach zum niederknien! …",abstand leckerst pizza halb abends mitnehmen bestellen positiv emotion einfach niederknien,5.0
4,Good place to dine in! Sadly no chicken pizzas available. Do keep in mind the waiting time when you visit✌🏻 …,chicken pizzas,4.0
5,This is the must try pizza in Frankfurt.,,5.0
6,5/5 Pizza place in Frankfurt,,5.0
7,Leckere Pizzen und coole Atmosphäre. 👍🏼 (,leck pizz cool atmosphäre,5.0


In [263]:
# renaming dataset columns, grabbing only the cleaned captions
df = df[['caption_clean', 'rating', 'timestamp']]
df = df.rename(columns={"caption_clean": "caption"})
df.reset_index(drop=True)

,caption,rating,timestamp
0,abstand leckerst pizza halb abends mitnehmen bestellen positiv emotion einfach niederknien,5.0,2023-02-15 22:10:07.146586
1,chicken pizzas,4.0,2023-02-14 00:10:07.146703
2,,5.0,2023-02-14 00:10:07.146705
3,,5.0,2023-02-14 00:10:07.146707
4,leck pizz cool atmosphäre,5.0,2023-02-13 00:10:07.146710
...,...,...,...
323,pizza empfehlen nett zuvorkommend service voll haus öfters pizza essen,5.0,2022-02-16 00:17:55.409988
324,leck pizza toll team,4.0,2022-02-16 00:17:55.410018
325,,5.0,2022-02-16 00:17:55.410048
326,favorite,5.0,2022-02-16 00:17:55.410078


In [264]:
# Remove null & empty values once again for 'caption_clean' column
df = df.loc[~df['caption'].isnull(), ['caption', 'rating', 'timestamp']]
df = df[df['caption'].str.strip().astype(bool)]

In [265]:
df['caption']

0      abstand leckerst pizza halb abends mitnehmen bestellen positiv emotion einfach niederknien
4                                                                                  chicken pizzas
7                                                                       leck pizz cool atmosphäre
11                                                                       extrem lecker empfehlung
13                                                                          gut pizza authentisch
                                                  ...                                            
493                                                                    essen super lecker service
495                        pizza empfehlen nett zuvorkommend service voll haus öfters pizza essen
496                                                                          leck pizza toll team
498                                                                                      favorite
499                 

In [266]:
df.to_csv('data/reviews-clean.csv', sep=';')